In [2]:
using Oscar
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [5]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [6]:
function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [7]:
function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return out
end
    

convertIncidenceMatrix (generic function with 1 method)

In [8]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 

coneListFormat (generic function with 1 method)

In [86]:
function toric_blowup(s, X, v)
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = rowMinors(coneList, starIndex)
    rayMatrix = X.RAYS
    
    #d = size(rayMatrix, 1) #length of array describing each ray
    clStar = []
    for t in star
        c = rank(rowMinors(rayMatrix, t)) - 1
        for i in t
            subcone = filter(elt -> elt != i, t)
            if rank(rowMinors(rayMatrix, subcone)) == c
                append!(clStar, [subcone])
            end
        end
    end
    clStar = unique(clStar)
                
    n = size(rayMatrix, 1)
    #coneList = filter(t -> !(t in star), coneList)
    coneList  = rowMinors(coneList, filter(i -> !(i in starIndex), range(1, stop=size(coneList, 1))))
    
    if size(s,1) == 1
        newCones = []
        for t in clStar
            if !(s[0] in t)
                append!(newCones, [sort(append!(t, s))])
            end
        end
        return fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            append!(t, [n])
        end
    end
    return fulton.NormalToricVariety(INPUT_RAYS = append!(Array(X.RAYS), [v]), INPUT_CONES = append!(coneList, newCones))
end

toric_blowup (generic function with 1 method)

In [ ]:
function toric_blowup(s,X)
    rays = rowMinors(Array(X.RAYS), s)
end

In [33]:
a = [[1 0], [0 1]]
rank(a)

LoadError: [91mMethodError: no method matching rank(::Array{Array{Int64,2},1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  rank([91m::Hecke.ModuleCtxNmod[39m) at /home/dhruvbhatia00/.julia/packages/Hecke/iivXP/src/Sparse/Module.jl:186[39m
[91m[0m  rank([91m::Hecke.ModuleCtx_fmpz[39m) at /home/dhruvbhatia00/.julia/packages/Hecke/iivXP/src/Sparse/Module.jl:182[39m
[91m[0m  rank([91m::GrpAbFinGen[39m) at /home/dhruvbhatia00/.julia/packages/Hecke/iivXP/src/GrpAb/GrpAbFinGen.jl:459[39m
[91m[0m  ...[39m

In [14]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [87]:
toric_blowup([1 2], tv, [1 1])

LoadError: [91mMethodError: no method matching svdvals!(::Polymake.MatrixAllocated{Float64})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  svdvals!([91m::LinearAlgebra.SymTridiagonal[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:350[39m
[91m[0m  svdvals!([91m::StridedArray{T, 2}[39m) where T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64} at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/svd.jl:193[39m
[91m[0m  svdvals!([91m::StridedArray{T, 2}[39m, [91m::StridedArray{T, 2}[39m) where T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64} at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/svd.jl:501[39m
[91m[0m  ...[39m

In [77]:
[[1 2], [2 3]]

2-element Array{Array{Int64,2},1}:
 [1 2]
 [2 3]

In [34]:
rank(Array(tv.RAYS))

2

In [48]:
filter(t -> t ==2, range(1,stop=5))

1-element Array{Int64,1}:
 2